In [1]:
#Step1: Importing necessary libraries
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#Step2: Mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
#Step3: Data Preprocessing and Cleaning for Turbidity Dataset.

import pandas as pd
# Load the dataset
data = pd.read_csv('/content/drive/MyDrive/Engg680_Project2024_Group15/dataset/Turbidity_cleaned data.csv')
# Clean the 'Sample Date' column by stripping spaces
data['Sample Date'] = data['Sample Date'].str.strip()
# Convert the 'Sample Date' column to datetime
# Adjust the format if you know it (e.g., '%m/%d/%Y' for MM/DD/YYYY)
data['Sample Date'] = pd.to_datetime(data['Sample Date'], errors='coerce')
# Check for invalid dates
if data['Sample Date'].isna().any():
  print("Warning: Some dates could not be parsed. Check these rows:")
  print(data[data['Sample Date'].isna()])
# Drop rows with invalid dates
data.dropna(subset=['Sample Date'], inplace=True)
# Set 'Sample Date' as the index
data.set_index('Sample Date', inplace=True)